# PortScan

In [8]:
from scapy.all import *
import ipaddress

ports = [25,80,53,443,445,8080,8443]

def SynScan(host):
    ans,unans = sr(
        IP(dst=host)/
        TCP(sport=33333,dport=ports,flags="S")
        ,timeout=2,verbose=0)
    print("Open ports at %s:" % host)
    for (s,r,) in ans:
        if s[TCP].dport == r[TCP].sport and r[TCP].flags=="SA":
            print(s[TCP].dport)

def DNSScan(host):
    ans,unans = sr(
        IP(dst=host)/
        UDP(dport=53)/
        DNS(rd=1,qd=DNSQR(qname="google.com"))
        
        ,timeout=2,verbose=0)
    if ans and ans[UDP]:
        print("DNS Server at %s"%host)
    
host = input("Enter IP Address: ")
try:
    ipaddress.ip_address(host)
except:
    print("Invalid address")
    exit(-1)

SynScan(host)
DNSScan(host)


Open ports at 8.8.8.8:
443
53
DNS Server at 8.8.8.8


In [1]:
!pip install scapy

     ---------------------------------------- 1.3/1.3 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444340 sha256=f54cc575a610b1aeec34f8b99e77c69e5362d0de46d25124db2806ad2e941059
  Stored in directory: c:\users\tan\appdata\local\pip\cache\wheels\8c\44\d2\b83890f39bfbc74d0cea746a58fae7e712bb3b78cc6b67d5d6
Successfully built scapy


# HoneyScan

In [9]:
from scapy.all import *

ip = "192.168.1.209"
ports = [53,80]
honeys = [8080,8443]

blocked = []

def analyzePackets(p):
    global blocked
    if p.haslayer(IP):
        response = Ether(src=p[Ether].dst,dst=p[Ether].src)/\
            IP(src=p[IP].dst,dst=p[IP].src)/\
            TCP(sport=p[TCP].dport,dport=p[TCP].sport,ack=p[TCP].seq+1)
        source = p[IP].src
    else:
        response = Ether(src=p[Ether].dst,dst=p[Ether].src)/\
            IPv6(src=p[IPv6].dst,dst=p[IPv6].src)/\
            TCP(sport=p[TCP].dport,dport=p[TCP].sport,ack=p[TCP].seq+1)
        source = p[IPv6].src
        
    port = p[TCP].dport
    if port in honeys:
        p.show()
    if source in blocked:
        if port in ports:
            response[TCP].flags = "RA"
        elif port in honeys: 
            response[TCP].flags = "SA"
        sendp(response,verbose=False)
    else:
        if source not in ports:
            blocked += source
            if port in honeys:
                response[TCP].flags = "SA"
                sendp(response,verbose=False)

f = "dst host "+ip+" and tcp"
sniff(filter=f,prn=analyzePackets)